Example of 'preprocess' step in which an input file describing the study region is discretized in H3 cells, visualized, and exported back.

In [ ]:
import geopandas
import h3
import folium
import matplotlib
import json
from shapely.geometry import Polygon
import importlib
import numpy

import sys
sys.path.append('../')

import DiscretizationBox
import DiscretizationBox.folium
import DiscretizationBox.travel_times.graphhopper


Import and visualize the original region:

In [ ]:
data_path = "YOUR_SHAPE_FILE"

In [ ]:
gdf = geopandas.read_file(data_path).reset_index()
gdf.plot()

Discretize it using the generate_discretization function. This uses Uber's H3 library function the find the H3 cells that better aproximate the study region in the given resolution.

In [ ]:
help(DiscretizationBox.generate_discretization)

In [ ]:
hex_gdf = DiscretizationBox.generate_discretization(gdf, shape = 'hexagons', h3_discretization_level = 6)
hex_gdf.plot()

The return is also a geodataframe, in the following format:

In [ ]:
hex_gdf.head(3)

We can visualize the discretization in more detail by using folium, for example

In [ ]:
DiscretizationBox.folium.visualize_h3(hex_gdf)

We can then export the discretization to file in all the formats supported by geopandas / fiona

In [ ]:
DiscretizationBox.save_gdf(hex_gdf, 'test', driver='ESRI Shapefile')

A list of all supported formats:

In [ ]:
import fiona; fiona.supported_drivers

For reimporting, we use load_gdf:

In [ ]:
t1 = DiscretizationBox.load_gdf('test/test.shp')
t1.head(3)

We can also obtain a distance matrix for these hexagons.

You must first set your graphhopper api key. Try not use all your credits at once! Be careful with large areas and/or small cells. 

In [ ]:
DiscretizationBox.travel_times.graphhopper.set_graphhopper_key("GH_KEY")

In [ ]:
dist_matrix = DiscretizationBox.travel_times.graphhopper.distance_matrix_from_gdf(hex_gdf)
dist_matrix

We can then cache it into a file, so we don't spend too many credits over time

In [ ]:
numpy.save('dist_matrix.npy'), dist_matrix, allow_pickle = True)

We can then read it back easily:

In [ ]:
dist_matrix_read = numpy.load('dist_matrix.npy', allow_pickle = True)
dist_matrix_read